In [199]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from functools import reduce
import pandas as pd
import pickle
from typing import List

In [200]:
spark = SparkSession.builder.appName('Exploracao').getOrCreate()

# Comentários

In [201]:
lista_assunto = [
    'power_bi',
    'python_and_dados',
    'cities_skylines',
    'cities_skylines_2',
    'linux',
    'linux_gamming',
    'genshin_impact',
    'zelda'
]

datas = [
    'extracao_data_2024_01_17',
    'extracao_data_2024_01_18',
    'extracao_data_2024_01_19',
    'extracao_data_2024_01_20',
    'extracao_data_2024_01_21',
    'extracao_data_2024_01_22',
    'extracao_data_2024_01_23',
]

In [202]:
df_comentarios = spark.read.json(f'../dados/bronze/assunto_power_bi/extracao_data_2024_01_17/comentarios/req_comentarios.json')

In [203]:
df_comentarios.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- canReply: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- isPublic: boolean (nullable = true)
 |    |    |    |-- topLevelComment: struct (nullable = true)
 |    |    |    |    |-- etag: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- kind: string (nullable = true)
 |    |    |    |    |-- snippet: struct (nullable = true)
 |    |    |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |    |    |-- authorChannelUrl: str

In [204]:
def abrir_comentarios( lista_datas: List, lista_assunto: List) -> List[DataFrame]:
    lista_df = []
    for assunto in lista_assunto:
        for datas in lista_datas:
            df_comentarios = spark.read.json(f'../dados/bronze/assunto_{assunto}/{datas}/comentarios/req_comentarios.json')
            
            dataframe = df_comentarios.select(
                    f.explode('items').alias('item'),
                    'data_extracao',
                    'etag',
                    'kind',
                    'pageInfo',
                    
                ).select(
                    'item.id',
                    'item.snippet.channelId',
                    'item.snippet.videoId',
                    'item.snippet.topLevelComment.snippet.textOriginal',
                    
                
            )
            lista_df.append(dataframe)
    return lista_df


In [205]:
lista_df = abrir_comentarios(lista_datas=datas, lista_assunto=lista_assunto)

In [206]:
df_final = reduce(lambda df1, df2: df1.union(df2), lista_df)
df_final.show(truncate=False)

24/02/29 21:40:36 WARN DAGScheduler: Broadcasting large task binary with size 1956.1 KiB
+--------------------------+------------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                        |channelId               |videoId    |textOriginal                                                                                            

In [207]:
df_comentarios_pandas = df_final.toPandas()
df_comentarios_pandas.head()

24/02/29 21:40:38 WARN DAGScheduler: Broadcasting large task binary with size 1943.6 KiB


,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise\nLIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...


In [208]:
df_comentarios_pandas[['id', 'channelId', 'videoId', 'textOriginal']] = df_comentarios_pandas[['id', 'channelId', 'videoId', 'textOriginal']].astype('string') 

In [222]:
df_comentarios_pandas = df_comentarios_pandas.drop_duplicates()

# Resposta Comentários

In [209]:
def abrir_respostas_comentarios( lista_datas: List, lista_assunto: List) -> List[DataFrame]:
    lista_df = []
    for assunto in lista_assunto:
        for datas in lista_datas:
            df_resposta_comentarios = spark.read.json(f'../dados/bronze/assunto_{assunto}/{datas}/resposta_comentarios/resposta_comentarios.json')
            
            dataframe = df_resposta_comentarios.select(
                f.explode('items').alias('item'),
                            
            ).select(
                'item.id',
                'item.snippet.channelId',
                'item.snippet.textOriginal',
                'item.snippet.parentId'
            )
            lista_df.append(dataframe)
    return lista_df

In [210]:
lista_df_resposta_comentarios = abrir_respostas_comentarios(lista_assunto=lista_assunto, lista_datas=datas)

In [211]:
df_resposta_comentarios_final = reduce(lambda df1, df2: df1.union(df2), lista_df_resposta_comentarios)
df_resposta_comentarios_final.show(truncate=False)

24/02/29 21:40:55 WARN DAGScheduler: Broadcasting large task binary with size 1210.1 KiB
+-------------------------------------------------+------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|id                                               |channelId               |textOriginal                                                                                                                                                                                                                                                                                                                                             |parentId                  |
+----------

In [212]:
df_resposta_comentarios_final_pandas = df_resposta_comentarios_final.toPandas()

24/02/29 21:40:57 WARN DAGScheduler: Broadcasting large task binary with size 1197.6 KiB


In [213]:
spark.stop()

In [215]:
df_resposta_comentarios_final_pandas[['id', 'channelId', 'textOriginal', 'parentId']] = df_resposta_comentarios_final_pandas[['id', 'channelId', 'textOriginal', 'parentId']].astype('string')

In [223]:
df_resposta_comentarios_final_pandas = df_resposta_comentarios_final_pandas.drop_duplicates()

# Concatenando dataframes de comentários

In [216]:
df_comentarios_pandas.head()

,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...


In [217]:
df_resposta_comentarios_final_pandas.head()


,id,channelId,textOriginal,parentId
0,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgPKKBrNVO,UCcuLEJUZy4jo9iFvngkoBUg,Há pelo menos dois,UgzZff_AwK5aYnauP6h4AaABAg
1,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgQElZCdaG,UCcuLEJUZy4jo9iFvngkoBUg,grato pelo retorno​@@SaladeEmprego,UgzZff_AwK5aYnauP6h4AaABAg
2,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgQFicpN39,UCcuLEJUZy4jo9iFvngkoBUg,grato pelo retorno​@@SaladeEmprego,UgzZff_AwK5aYnauP6h4AaABAg
3,UgxIk3_tEF6SSnEjGDV4AaABAg.9zgB2WyYMq69zgPIGQSki6,UCcuLEJUZy4jo9iFvngkoBUg,Sucesso!,UgxIk3_tEF6SSnEjGDV4AaABAg
4,UgzQwjfcCgBDQVEgCkt4AaABAg.9zfoXBRb8Lb9zgBDjFoOcm,UCcuLEJUZy4jo9iFvngkoBUg,Normalmente a entrevista em grupo costuma ser ...,UgzQwjfcCgBDQVEgCkt4AaABAg


In [220]:
df_comentarios_completos = pd.merge(
    left=df_comentarios_pandas, 
    right=df_resposta_comentarios_final_pandas,
    how='left',
    left_on='id',
    right_on='parentId'
)

In [221]:
df_comentarios_completos.head()

,id_x,channelId_x,videoId,textOriginal_x,id_y,channelId_y,textOriginal_y,parentId
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.,UgwBml2VBGvh47H6CgZ4AaABAg.9zfMad51A7T9zfT8vqYrxG,UCENbpaPgVhO2GtlNcwAJ8mA,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast...",UgwBml2VBGvh47H6CgZ4AaABAg
1,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.,UgwBml2VBGvh47H6CgZ4AaABAg.9zfMad51A7T9zfT8vqYrxG,UCENbpaPgVhO2GtlNcwAJ8mA,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast...",UgwBml2VBGvh47H6CgZ4AaABAg
2,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.,UgwBml2VBGvh47H6CgZ4AaABAg.9zfMad51A7T9zfT8vqYrxG,UCENbpaPgVhO2GtlNcwAJ8mA,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast...",UgwBml2VBGvh47H6CgZ4AaABAg
3,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.,UgwBml2VBGvh47H6CgZ4AaABAg.9zfMad51A7T9zfT8vqYrxG,UCENbpaPgVhO2GtlNcwAJ8mA,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast...",UgwBml2VBGvh47H6CgZ4AaABAg
4,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.,UgwBml2VBGvh47H6CgZ4AaABAg.9zfMad51A7T9zfT8vqYrxG,UCENbpaPgVhO2GtlNcwAJ8mA,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast...",UgwBml2VBGvh47H6CgZ4AaABAg


# Abrindo dataframe de canal, vídeo